**Name:** Bala Guga Gopal S

**Script Name:**  3.2. Text Auto Completion Data

**Script Description:** Text auto generation using RNN(LSTM) from the previous word

# Contents

1. Dataset download
2. Import libraries
3. Load the dataset
4. VISUALAZING THE DATASET
4. DATASET Preprocessing
5. RNN model
6. Train the model
7. Predictiom
8. Evaluation

#  Dataset download

The data contains information about the comments made on the articles published in New York Times in Jan-May 2017 and Jan-April 2018. The month-wise data is given in two csv files - one each for the articles on which comments were made and for the comments themselves. The csv files for comments contain over 2 million comments in total with 34 features and those for articles contain 16 features about more than 9,000 articles.

LINK - https://www.kaggle.com/datasets/aashita/nyt-comments?resource=download

In [ ]:
# Configure Working directory to for Kaggle Json image (important to download the dataset from Kaggle to google drive via colab)
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d aashita/nyt-comments

 98% 472M/480M [00:03<00:00, 168MB/s]
100% 480M/480M [00:03<00:00, 158MB/s]


In [ ]:
# Unzip the folders and to delete the .zip file.
!unzip \*.zip  && rm *.zip

Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


# Import libraries


In [ ]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils
import keras.utils as ku 

# set seeds for reproducability
import tensorflow as tf
from numpy.random import seed
tf.random.set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the dataset


In [ ]:
working_dir = '/content/'
headlines = []

for filename in os.listdir(working_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(working_dir + filename)
        headlines.extend(list(article_df.headline.values))

headlines = [x for x in headlines if x!="Unknown"]
len(headlines)

8603

# VISUALAZING THE DATASET

In [ ]:
# Checking out one dataset

testing_ar = pd.read_csv('/content/ArticlesApril2017.csv')
testing_ar.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [ ]:
# Checking out the comments
testing_cm = pd.read_csv('/content/CommentsApril2017.csv')
testing_cm.head()

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


# Dataset preprocessing




## TOKENIZATION
All the words and their indices will be stored in a dictionary which you can access it using tokenizer.word_index. Therefore, you can find the number of the unique words based on the number of elements in this dictionary:

num_words = len(tokenizer.word_index) + 1

That + 1 is because of reserving padding (i.e. index zero).

In [ ]:
sample = headlines[:10]
sample

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree']

In [ ]:
# Basic iteration over each headline
[x for x in sample]

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree']

In [ ]:
# Nested comprehesion for iteration over each element
[y for x in sample[0] for y in x]

['N',
 '.',
 'F',
 '.',
 'L',
 '.',
 ' ',
 'v',
 's',
 '.',
 ' ',
 'P',
 'o',
 'l',
 'i',
 't',
 'i',
 'c',
 's',
 ' ',
 'H',
 'a',
 's',
 ' ',
 'B',
 'e',
 'e',
 'n',
 ' ',
 'B',
 'a',
 't',
 't',
 'l',
 'e',
 ' ',
 'A',
 'l',
 'l',
 ' ',
 'S',
 'e',
 'a',
 's',
 'o',
 'n',
 ' ',
 'L',
 'o',
 'n',
 'g']

In [ ]:
def cleaning(data):
    cleaned = "".join(x for x in data if x not in string.punctuation).lower()
    cleaned = cleaned.encode("utf8").decode("ascii", "ignore")
    return cleaned

cleaned_l = [cleaning(x) for x in headlines]
cleaned_l[:10]

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree']

## Generating Sequence of N-gram Tokens
Language modelling requires a sequence input data, as given a sequence (of words/tokens) the aim is the predict next word/token.

The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens.

In [ ]:
num_words = len(tokenizer.word_index) + 1
sample = cleaned_l[:2]
sample

['nfl vs politics has been battle all season long', 'voice vice veracity']

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(sample)
tk.word_index

{'all': 7,
 'battle': 6,
 'been': 5,
 'has': 4,
 'long': 9,
 'nfl': 1,
 'politics': 3,
 'season': 8,
 'veracity': 12,
 'vice': 11,
 'voice': 10,
 'vs': 2}

In [ ]:
tk.texts_to_sequences([sample[0]])

[[1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [ ]:
tk.word_index

{'all': 7,
 'battle': 6,
 'been': 5,
 'has': 4,
 'long': 9,
 'nfl': 1,
 'politics': 3,
 'season': 8,
 'veracity': 12,
 'vice': 11,
 'voice': 10,
 'vs': 2}

In [ ]:
len(tk.word_index)

12

In [ ]:
sample = cleaned_l[:5]
sample

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean']

In [ ]:
for line in sample:
    print(line[0])

n
v
a
n
a


In [ ]:
tk = Tokenizer()

def text_tokens(data):
    tk.fit_on_texts(data)
    # Our vocabulary size will be +1 due to incorporation of padding, which is treated like another word
    total_vocab = len(tk.word_index)+1
    input_sequences=[]
    
    for record in data:
        # This will give a 2D array, the first element will be the token list
        token_list = tk.texts_to_sequences([record])[0]
        for i in range(1, len(token_list)):
            # Append first two elements, then 3, then 4, then 5
            n_gram = token_list[:i+1]
            input_sequences.append(n_gram)
    return input_sequences, total_vocab

inp_seq, vocab_size = text_tokens(cleaned_l)
inp_seq[:10]

[[915, 148],
 [915, 148, 212],
 [915, 148, 212, 73],
 [915, 148, 212, 73, 475],
 [915, 148, 212, 73, 475, 202],
 [915, 148, 212, 73, 475, 202, 68],
 [915, 148, 212, 73, 475, 202, 68, 28],
 [915, 148, 212, 73, 475, 202, 68, 28, 182],
 [436, 3366],
 [436, 3366, 5166]]

## Padding sequence

A uniform length of all the input vectors is created, that can be passed in the input layer of our LSTM model. We will also separate out the predictor features from the target labels.

The features will be all values from the start, till the 2nd last value
The target will be the final value in the given sequence



In [ ]:
s = pd.Series(inp_seq)

In [ ]:
max_length = np.max(s.apply(lambda x: len(x)))
max_length

24

In [ ]:
def padding(inp):
    max_seq_len = max_length
    inp_seq = np.array(pad_sequences(inp, maxlen=max_seq_len, padding='pre'))
    features, target = inp_seq[:, :-1], inp_seq[:, -1]
    target = ku.np_utils.to_categorical(target, num_classes=vocab_size)
    return features, target, max_seq_len

features, target, max_seq_len = padding(inp_seq)

In [ ]:
features.shape

(51770, 23)

In [ ]:
target.shape

(51770, 11265)

#RNN Model

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively. To learn more about LSTMs, here is a great post. Lets architecture a LSTM model in our code. I have added total three layers in the model.

Input Layer : Takes the sequence of words as input
LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.
Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer)
Output Layer : Computes the probability of the best possible next word as output
We will run this model for total 100 epoochs but it can be experimented further.

In [ ]:
model=Sequential()
# We define max length - 1 as now our features have one column less (used as target label)
model.add(Embedding(vocab_size, 10, input_length=max_seq_len-1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 23, 10)            112650    
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 11265)             1137765   
                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________


# Train the Model

In [ ]:
history = model.fit(features, target, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

# Prediction

In [ ]:
def predictions(seed_text, next_words, model, max_seq):
    for _ in range(next_words):
        # Conver the seed text into tokens
        token_list = tk.texts_to_sequences([seed_text])[0]
        # Pad the seed text with seq-1 (as we only have features)
        token_list = pad_sequences([token_list], maxlen=max_seq-1, padding='pre')
        # Predict the class of the next word
        predicted = np.argmax(model.predict(token_list),axis=1)        
        
        outputs = ""
        # Iterate over the dictionary, to get the word
        for word, index in tk.word_index.items():
            if index==predicted:
                output_word = word
                break
        # Add the word in the seed text, for the next prediction
        seed_text = seed_text+" "+output_word
    return seed_text.title()

In [ ]:
print (predictions("united states", 5, model, max_seq_len))
print (predictions("preident trump", 4, model, max_seq_len))
print (predictions("donald trump", 4, model, max_seq_len))
print (predictions("india and new york", 4, model, max_seq_len))
print (predictions("new york", 4, model, max_seq_len))
print (predictions("science ", 5, model, max_seq_len))

United States Says Big Leash As Technology
Preident Trump Is Poised To Expose
Donald Trump Vs The Food Snobs
India And New York Times Capitalism My Travel
New York Today Our Locals On
Science  Panel Cites Dangers In Vaping
